## ETL01_Stock_price

In [91]:
from pyspark.sql.types import *
from pyspark.sql import functions as f
import requests
import pandas as pd 
from datetime import datetime, timedelta
from notebookutils import mssparkutils

# get data from api 
def get_stock_data(symbol,time_from,data_size,apikey):
    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&symbol={0}&time_from={1}&limit={2}&apikey={3}'.format(symbol,time_from,data_size,apikey)
    r = requests.get(url)
    data = r.json()
    return data

def transform_pdf(news_list):
    dfs=[]
    for news in news_list:
        df = pd.DataFrame.from_dict(news['feed'])
        dfs.append(df)
    pdf = pd.concat(dfs).drop_duplicates(subset=['title'])
    pdf_v1 = pdf[['title','time_published','source','ticker_sentiment']]
    return pdf_v1

StatementMeta(spark001, 14, 92, Finished, Available)

In [87]:
if __name__ == "__main__":
    # get apikey from key-vault 
    apikey = mssparkutils.credentials.getSecret('synapse-key123','alphavantage-key')
    data_size=200
    # set up target ticker 
    symbol_list=['NVDA','AMD','INTC','QCOM','GOOG','MSFT','AMZN','AAPL']
    # Get the current datetime
    now = datetime.now()

    # 30 days to the current datetime
    future_date = now - timedelta(days=30)

    # Format both datetime objects as 'YYYYMMDDTHHMM'
    formatted_now = now.strftime('%Y%m%dT%H%M')
    time_from = future_date.strftime('%Y%m%dT%H%M')
    # get data from api
    stock_news = [get_stock_data(symbol,time_from,data_size,apikey) for symbol in symbol_list]
    pdf = transform_pdf(stock_news)
    sp_df = spark.createDataFrame(pdf).filter(f.size(f.col('ticker_sentiment')) > 0)

    sp_df_v1 = (
        sp_df.select('title','time_published','source',f.explode('ticker_sentiment').alias("ticker_sentiment"))
        .select('title','time_published','source','ticker_sentiment.*')
        .filter(f.col('ticker').isin(symbol_list))
                    )

StatementMeta(spark001, 14, 88, Finished, Available)

In [99]:
    print('load data into filepath')
    #add year column  
    file_path ='abfss://files@datalake9vbgk2l.dfs.core.windows.net/synapse/workspaces/data/stock_news'
    sp_df_v1.write.option("header", True).csv(file_path,mode='overwrite')

StatementMeta(spark001, 14, 100, Finished, Available)

load data into filepath
